In [36]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr, wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

AUCs.txt
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [37]:
aucs = pd.read_csv('AUCs.txt', sep = '\t', header = 0)
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [38]:
aucs = aucs.drop(["Unnamed: 0"], axis = 1)

In [39]:
corr_data = []
for i, col1 in enumerate(aucs.columns):
    for j, col2 in enumerate(aucs.columns):
        if j <= i:
            continue
        corr, p = wilcoxon(aucs[col1], aucs[col2])
        corr_data.append([col1, col2, corr, p])

In [40]:
corr_data = pd.DataFrame(corr_data)
corr_data.columns = ['Method_1','Method_2','corr','p']

In [49]:
corr_data.sort_values(by='p', ascending=True)

,Method_1,Method_2,corr,p
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015906
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [47]:
(corr_data.p < 0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

Answer: C4.5+m

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [52]:
from collections import Counter

reject, p_corrected, a1, a2 = multipletests(corr_data.p, 
                                            alpha = 0.05, 
                                            method = 'holm')
Counter(reject)

Counter({False: 6})

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [53]:
reject, p_corrected, a1, a2 = multipletests(corr_data.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
Counter(reject)

Counter({False: 3, True: 3})